In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!ls

datalab  drive


In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [5]:
import keras
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

Using TensorFlow backend.


In [0]:
data_path = 'drive/projects/QuoteGen/data'
if not os.path.exists(data_path):
  os.mkdir(data_path)

In [0]:
models_path = 'drive/projects/QuoteGen/models'
if not os.path.exists(models_path):
  os.mkdir(models_path)

In [0]:
data_txt = os.path.join(data_path, 'quotes.txt')
data_csv = os.path.join(data_path, 'quotes_all.csv')

In [9]:
quotes_ds = []
if os.path.exists(os.path.join(data_path,"quotes_pickle.pkl")):
  with open(os.path.join(data_path,"quotes_pickle.pkl"),'rb') as f:
    quotes_ds = pickle.load(f)
else:
  with open(data_csv) as f:
    for line in f.readlines():
      quotes_ds.append(line.split(";")[0])
  # Pickling the dataset
  if not os.path.exists(os.path.join(data_path,'quotes_pickle.pkl')):
    out = open(os.path.join(data_path,"quotes_pickle.pkl"),'wb')
    pickle.dump(quotes_ds, out)
print("Found %d quotes"%len(quotes_ds))

Found 75966 quotes


In [10]:
quotes_ds[0]

"Age is an issue of mind over matter. If you don't mind, it doesn't matter."

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
n_words = 500
max_seq_len = 30

In [39]:
tokenizer = Tokenizer(n_words)
tokenizer.fit_on_texts(quotes_ds)
sequences = tokenizer.texts_to_sequences(quotes_ds)

word_index = tokenizer.word_index
print('Found %d unique tokens.' % len(word_index))

Found 34015 unique tokens.


In [0]:
data = pad_sequences(sequences, maxlen=max_seq_len, padding = 'post')

In [0]:
index2word = {}
for word in tokenizer.word_index:
  index2word[tokenizer.word_index.get(word)] = word

In [58]:
test = data[1]
s = ""
for temp in test:
  if not temp==0:
    s = s+ " "+ index2word[temp]
    
quotes_ds[1],s

('Anyone who stops learning is old, whether at twenty or eighty. Anyone who keeps learning stays young. The greatest thing in life is to keep your mind young.',
 ' anyone who learning is old whether at or anyone who learning young the greatest thing in life is to keep your mind young')

In [0]:
# TODO:
# Shift data to create target data

In [0]:
def shift(arr,num):
    arr=np.roll(arr,num)
    if num<0:
         np.put(arr,range(len(arr)+num,len(arr)),0)
    elif num > 0:
         np.put(arr,range(num),0)
    return arr

In [0]:
target_data = shift(data,1)

In [62]:
data[0], target_data[0]

(array([166,   7,  38,   4, 248, 168, 336,  30,  11,  50, 248,  10, 264,
        336,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0], dtype=int32),
 array([  0, 166,   7,  38,   4, 248, 168, 336,  30,  11,  50, 248,  10,
        264, 336,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0], dtype=int32))

In [0]:
def generate_data(data, lookback,shuffle=False, batch_size=1,delay = 1):
  samples = np.zeros((batch_size, lookback))
  targets = np.zeros((batch_size,1))
  min_index = lookback
  max_index = max_seq_len - delay -1
  
  while 1:
    for i in range(batch_size):
      index_sent = np.random.randint(0,len(data))
      sentence = np.array(data[index_sent])
      if(len(sentence)-lookback>lookback):
        index_word = np.random.randint(min_index,len(sentence)-delay -1)
        indices = range(index_word-lookback, index_word)
      else:
        continue
      samples[i,:] = sentence[indices]
      targets[i] = sentence[index_word+1]
    yield samples, targets

In [0]:
from keras.models import Sequential
from keras import Model
from keras.layers import Input, Embedding, LSTM, Dense, Flatten,Dropout
from keras.optimizers import RMSprop

In [0]:
model = Sequential()
model.add(Embedding(n_words, 64, input_length=lookback))
# model.add(Flatten())
model.add(LSTM(256,return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(256,activation='relu'))
model.add(Dense(1))

In [0]:
model.compile(loss='mae', optimizer='rmsprop')

In [0]:
lookback = 10
delay = 1
step = 1
batch_size = 128
epochs = 50

In [0]:
train_gen = generate_data(sequences, lookback= lookback,shuffle=False, batch_size=batch_size,delay = 1)
val_gen = generate_data(sequences, lookback= lookback,shuffle=False, batch_size=batch_size,delay = 1)

In [0]:
history = model.fit_generator(temp,steps_per_epoch=500, epochs=epochs, validation_data=val_gen, validation_steps=10)

Epoch 1/50
500/500 [==============================] - 38s 77ms/step - loss: 73.1885 - val_loss: 75.8926
Epoch 2/50
330/500 [==================>...........] - ETA: 12s - loss: 71.9196

500/500 [==============================] - 36s 72ms/step - loss: 72.1726 - val_loss: 80.9955
Epoch 3/50
500/500 [==============================] - 35s 71ms/step - loss: 72.1091 - val_loss: 70.9875
Epoch 4/50
 10/500 [..............................] - ETA: 34s - loss: 71.9893

406/500 [=======================>......] - ETA: 6s - loss: 71.6245